# Get data and label data-frame from excel file

Save a tuble of 'x' and 'y' into jupyter space.

#### Input
1. NA

#### Output 
1. %store data_label 


### Usage on the caller notebook
    %store -r data_label
    x = data_label[0]  # features ndarray[numpy.float64... X 751]
    y = data_label[1]  # labels ndarray[numpy.float64]
    
### Examine the data

    [x] 把 data frame 倒成 csv 的方法，用來檢查資料
        df.to_csv(file_name, sep='\t', encoding='utf-8')
    [x] 應用 https://pair-code.github.io/facets/ 可看出很多問題。  

    

In [1]:
import pandas as pd
import numpy as np
import peforth

In [2]:
df = pd.read_csv('000777.csv',encoding='gbk',parse_dates=[0],index_col=0)

In [3]:
# pdb.set_trace() # data 本來是新的在上面， sort 過後變成最舊的在上面
df.sort_index(0,ascending=True,inplace=True) 

In [4]:
# data frame 中帶有 None 的 rows 都剔除掉。 
data = df.replace(to_replace='None', value=np.nan).dropna()  

In [5]:
# 上市第一天的資料不要，因為【漲跌幅】無效。
data = data.query('index > "1997-07-10"')

In [6]:
# raw data has 20 years long, but here only analysis the last 150 days of a chosen day
dayfeature=150
dayforesee=5  # 問未來幾天如何如何，最後就要扣掉幾天，避免 out of bounds. 0 代表當天。
featurenum=5*dayfeature  # 取五個有關係的欄位，特徵值。
x=np.zeros((data.shape[0]-dayfeature-dayforesee,featurenum+1))
y=np.zeros((data.shape[0]-dayfeature-dayforesee));

In [7]:
# 重疊 shift 150 天的五個「行情數目」加上下一天的開盤 共 751 個「數目」當作 feature X 

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    x[i,0:featurenum]=np.array(data[i:i+dayfeature] \
          [[u'收盘价',u'最高价',u'最低价',u'开盘价',u'成交量']]).reshape((1,featurenum))
    # x[i,featurenum]=data.ix[i+dayfeature][u'开盘价']  # ix deprecated
    x[i,featurenum]=data.iloc[i+dayfeature][u'开盘价']

# 這裡設定要問 AI 的題目

In [8]:
# 預測未來 5 天是否會「漲20%」?

# data.shape[0]-dayfeature 是 4489 同上 x, 所以這是在製作相對所有 x 的 y. x 是 (4488, 751)
# dayfeature 是 150 
# 

for i in range(0,data.shape[0]-dayfeature-dayforesee):
    y[i]=0              
    pi = float(data.iloc[i][u'收盘价'])
    for j in range(dayforesee):
        pi5 = float(data.iloc[j+i+dayfeature][u'最高价'])
        if (pi5-pi)/pi >= 0.20 :
            y[i]=1


In [36]:
# 核對一下
%f __main__ :> x.shape ." x shape " . cr
%f __main__ :> x dup type ." x type " . :> [0][0] type ." , x[0][0] type " . cr    
%f __main__ :> y.shape ." y shape " . cr
%f __main__ :> y dup type ." y type " . :> [0]    type ." , y[0] type " . cr    
%f __main__ :> x cr ." x: " . cr
%f __main__ :> y cr ." y: " . cr    
%f __main__ :> y py> sum(pop()) ." sum of y : " . cr    


x shape (4483, 751)
x type <class 'numpy.ndarray'>, x[0][0] type <class 'numpy.float64'>
y shape (4483,)
y type <class 'numpy.ndarray'>, y[0] type <class 'numpy.float64'>

x: [[9.9700000e+00 9.9700000e+00 8.4800000e+00 ... 1.1990000e+01
  1.9075000e+05 1.1650000e+01]
 [1.0970000e+01 1.0970000e+01 1.0330000e+01 ... 1.1650000e+01
  4.1790000e+05 1.1000000e+01]
 [1.1530000e+01 1.1800000e+01 1.0810000e+01 ... 1.1000000e+01
  2.4310400e+05 1.1150000e+01]
 ...
 [2.2380000e+01 2.2670000e+01 2.2200000e+01 ... 2.3400000e+01
  1.0402149e+07 2.3220000e+01]
 [2.2630000e+01 2.2660000e+01 2.2310000e+01 ... 2.3220000e+01
  1.5360483e+07 2.3410000e+01]
 [2.2970000e+01 2.3090000e+01 2.2410000e+01 ... 2.3410000e+01
  1.0800996e+07 2.2710000e+01]]

y: [0. 0. 0. ... 0. 0. 0.]
sum of y : 1571.0


In [35]:
data_label = (x,y)
%store data_label

Stored 'data_label' (tuple)


### 9. IPython Magic - %store: Pass variables between notebooks.

The %store command lets you pass variables between two different notebooks.

    data = 'this is the string I want to pass to different notebook'
    %store data
    del data # This has deleted the variable

    Stored 'data' (str)

Now, in a new notebook...

    %store -r data
    print(data)

    this is the string I want to pass to different notebook